In [ ]:
#|default_exp wandb

# Weights & Biases Sweeps

>Weights & Biases Sweeps are used to automate hyperparameter optimization and explore the space of possible models.

In [ ]:
#|export
import warnings

from fastcore.script import *

from tsai.export import *
from tsai.imports import *
from tsai.utils import *

In [ ]:
#|export
def wandb_agent(script_path, sweep, entity=None, project=None, count=None, run=True):
    "Run `wandb agent` with `sweep` and `script_path"
    try: import wandb
    except ImportError: raise ImportError('You need to install wandb to run sweeps!')
    if 'program' not in sweep.keys(): sweep["program"] = script_path
    sweep_id = wandb.sweep(sweep, entity=entity, project=project)
    entity = ifnone(entity, os.environ['WANDB_ENTITY'])
    project = ifnone(project, os.environ['WANDB_PROJECT'])
    print(f"\nwandb agent {entity}/{project}/{sweep_id}\n")
    if run: wandb.agent(sweep_id, function=None, count=count)

get_wandb_agent = named_partial("get_wandb_agent", wandb_agent, run=False)

run_wandb_agent = named_partial("run_wandb_agent", wandb_agent, run=True)

In [ ]:
#|export
def update_run_config(config, new_config, verbose=False):
    "Update `config` with `new_config`"
    config_dict = config.copy()
    if hasattr(config_dict, "sweep"):
        del config_dict["sweep"]
    for k, v in new_config.items():
        if k in config_dict.keys():
            if verbose:
                print(f"config.{k} {config_dict[k]} updated to {new_config[k]}")
            config_dict[k] = new_config[k]
        elif (
            hasattr(config_dict, "arch_config")
            and k in config_dict["arch_config"].keys()
        ):
            if verbose:
                print(
                    f"config.arch_config.{k} {config['arch_config'][k]} updated to {new_config[k]}"
                )
            config["arch_config"][k] = new_config[k]
        else:
            warnings.warn(f"{k} not available in config or config.arch_config")
    return config_dict

In [ ]:
#|export
def get_sweep_config(config):
    "Get sweep config from `config`"
    if not hasattr(config, "sweep") or not config["sweep"]: 
        return {}
    if isinstance(config["sweep"], str):
        file_path = Path(config["sweep"])
        file_path = to_root_path(file_path)
        sweep = yaml2dict(file_path, attrdict=False)
    else:
        sweep = attrdict2dict(config["sweep"])
    if hasattr(sweep, "sweep"):
        sweep = sweep["sweep"]
    return sweep

In [ ]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

/Users/nacho/notebooks/tsai/nbs/073_wandb.ipynb saved at 2023-03-07 19:53:26
Correct notebook to script conversion! 😃
Tuesday 07/03/23 19:53:29 CET
